In [1]:
import re
import pandas as pd
import csv
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
with open('../data/log.txt', encoding='utf-8') as f:
    lines = f.readlines()
commits = []
i = 0
while i < len(lines):
    line = lines[i]
    if line.startswith("commit"):
        commit = []
        commit.append(line)
        j = i+1
        while (j < len(lines) and (not lines[j].startswith("commit"))):
            commit.append(lines[j])
            j+= 1
        commits.append(commit)
        i = j
len(commits)

10537

In [3]:
EXPECTATIONS_ENUM = [' Timeout ',
 ' Skip ',
 ' Pass ',
 ' Failure Pass ',
 ' Failure ',
 ' Crash ',
 ' Failure Timeout ',
 ' Pass Timeout ',
 ' Timeout Pass ',
 ' Crash Failure Pass Timeout ',
 ' Failure Pass Timeout ',
 ' Failure Pass Skip Timeout ',
 ' Skip Timeout ',
 ' Pass Timeout Failure ',
 ' Pass Failure Timeout ',
 ' Pass Failure Timeout Skip ',
 ' Pass Failure ',
 ' Crash Timeout ',
 ' Crash Pass ',
 ' Pass Skip Timeout ',
 ' Crash Failure Pass ',
 ' Crash Timeout Skip Failure Pass ',
 ' Crash Failure Pass Skip Timeout ',
 ' Crash Skip Timeout ',
 ' Crash Failure ',
 ' Crash Pass Timeout ',
 ' Skip Timeout Failure ',
 ' Failure Skip Timeout ',
 ' Failure Crash ',
 ' Skip Pass Failure Timeout ',
 ' Crash Pass Failure ',
 ' TIMEOUT ',
 ' Timeout Failure Pass ',
 ' Pass Crash ',
 ' Timeout Failure ',
 ' Crash Failure Timeout ',
 ' Crash Pass Skip Timeout ',
 ' Failure Timeout Skip ',
 ' Failure Pass Crash Timeout ',
 ' Failure Pass Crash ',
 ' Pass Skip ',
 ' Timeout Pass Failure ',
 ' Pass Failure Crash Timeout ',
 ' Pass Failure Crash ',
 ' Timeout Crash Pass Failure Skip ',
 ' Pass Crash Timeout ',
 ' Crash Pass Failure Timeout ',
 ' Failure Crash Pass Timeout ',
 ' Timeout Crash ',
 ' Failure Crash Pass ',
 ' Pass Crash Failure ',
 ' Failure Pass Timeout Crash ',
 ' Failure Timeout Pass ',
 ' Crash Timeout Pass ',
 ' Pass Timeout Crash ',
 ' Pass Failure Timeout Crash ',
 ' Pass Timeout Crash Failure ',
 ' Crash Timeout Failure ',
 ' SKip ',
 '  Failure ',
 ' Failure Crash Timeout ',
 ' Timeout Failure Crash Pass ',
 ' Skip  ',
 ' Timeout Crash Failure Pass ',
 ' Failure Timeout Crash ',
 '  ']

In [4]:
df = pd.DataFrame(columns=['Commit_hash','Author','Date', 'Description', 'Bug_Id', 'Change_Id', 'Reviewed-on','Test', 'Expectation', 'Build_type', 'Action'])

In [ ]:
counter = 0
for commit in commits:
#     print("**********************************************************************************")
    change_ids = ""
    bug_id = ''
    descriptoin = ''
    for index, line in enumerate(commit):
        test = ''
        build_type = ''
        if line.startswith("commit"):
            commit_hash = line.replace("commit ", "")
            commit_hash = commit_hash.replace("/n ", "")
        elif line.startswith("Author:"):
            author = line.replace("Author: ", "")
            author = author.replace("/n ", "")
        elif line.startswith("Date"):
            date = line.replace("Date: ", "")
            date = date.replace(" +0000\n", "")
            description_start = index + 1
        elif line.startswith("diff"):
            description_end = index
            descriptoin_arr = commit[description_start:description_end]
            descriptoin = ' '.join([str(w) for w in descriptoin_arr])
            descriptoin = descriptoin.replace("/n ", "")
        elif line.startswith("    Bug: "):
            bug_id = line.replace("    Bug: ", "")
            bug_id = bug_id.replace("/n ", "")
        elif line.startswith("    Reviewed-on: "):
            reviewed_on = line.replace("    Reviewed-on: ", "")
            reviewed_on = reviewed_on.replace("/n ", "")
        elif line.startswith("    Change-Id:"):
            change_id = line.replace("    Change-Id: ", "")
            change_id = change_id.replace("/n ", "")
            change_ids += change_id + " "
        elif line.startswith("-crbug.com"):
            counter += 1
            line = line.replace(" /n", "")
            removed_expectation = line[line.find("[")+1 : line.find("]")]
            
            # for the case line has [build_type ex: mac10] in it, we need to capture second [] pattern
            if removed_expectation not in EXPECTATIONS_ENUM:
                build_type = removed_expectation
                new_line = line[line.find("]") + 1:]
                removed_expectation = new_line[new_line.find("[")+1 : new_line.find("]")]
                test = new_line[:new_line.find("[")]
            if test == '':
                x = line.split(" ")
                test = x[1]
            expectation_arr = removed_expectation.split()
            expectation_arr = sorted(expectation_arr)
            removed_expectation = ' '.join([str(w.lower()) for w in expectation_arr])
            test_row = {'Commit_hash': commit_hash,
                        'Author': author,
                        'Date': date,
                        'Description':descriptoin,
                        'Bug_Id': bug_id,
                        'Change_Id': change_ids,
                        'Reviewed-on': reviewed_on,
                        'Test': test,
                        'Expectation': removed_expectation,
                        'Build_type': build_type,
                        'Action': 'remove'}
            df = df.append(test_row, ignore_index = True)
        elif line.startswith("+crbug.com"):
            counter += 1
            line = line.replace("/n ", "")
            added_expectation = line[line.find("[")+1 : line.find("]")]
            if added_expectation not in EXPECTATIONS_ENUM:
                build_type = added_expectation
                new_line = line[line.find("]") + 1:]
                added_expectation = new_line[new_line.find("[")+1 : new_line.find("]")]
                test = new_line[:new_line.find("[")]
            if test == '':
                x = line.split(" ")
                test = x[1]
            expectation_arr = added_expectation.split()
            expectation_arr = sorted(expectation_arr)
            added_expectation = ' '.join([str(w.lower()) for w in expectation_arr])
            test_row = {'Commit_hash': commit_hash,
                        'Author': author,
                        'Date': date,
                        'Description':descriptoin,
                        'Bug_Id': bug_id,
                        'Change_Id': change_ids,
                        'Reviewed-on': reviewed_on,
                        'Test': test,
                        'Expectation': added_expectation,
                        'Build_type': build_type,
                        'Action': 'add'}
            df = df.append(test_row, ignore_index = True)

df.to_csv('full_log_df.csv')

In [ ]:
test_df = pd.read_csv('full_log_df.csv')
added_test = test_df.copy()
added_test = added_test[added_test['Action'] == 'add']
for i in range(len(added_test)):
    added_test.iloc[i, added_test.columns.get_loc('Date')] = datetime.strptime(added_test.iloc[i].Date, '  %a %b %d %H:%M:%S %Y').date()
added_test = added_test.iloc[::-1]

removed_test = test_df.copy()
removed_test = removed_test[removed_test['Action'] == 'remove']
for i in range(len(removed_test)):
    removed_test.iloc[i, removed_test.columns.get_loc('Date')] = datetime.strptime(removed_test.iloc[i].Date, '  %a %b %d %H:%M:%S %Y').date()
removed_test = removed_test.iloc[::-1]

life_time_df = added_test
temp_removed = removed_test
life_time_df['Life_Time'] = -1
for i in range(len(added_test)):
    print(added_test['Date'].iloc[i])
    add_date = added_test['Date'].iloc[i]
    
    temp_removed = temp_removed[temp_removed.Date >= added_test['Date'].iloc[i]]
    for j in range(len(temp_removed)):
        if added_test['Test'].iloc[i] == temp_removed['Test'].iloc[j]:
            remove_date = temp_removed['Date'].iloc[j]
            life_time = (remove_date - add_date).days
            if life_time < 0:
                life_time = -1
            print(life_time, remove_date)
            life_time_df['Life_Time'].iloc[i] = life_time
            temp_removed = temp_removed.drop(temp_removed.index[j])
            break

life_time_df.to_csv('Life_time_full_log.csv')

In [ ]:
removed_flaky = flaky_df[flaky_df['Action'] == 'remove']

life_time_df = added_flaky
temp_remove_flaky = removed_flaky
life_time_df['Life_Time'] = -1
for i in range(len(added_flaky)):
    print(added_flaky['Date'].iloc[i])
    add_date = datetime.strptime(added_flaky['Date'].iloc[i], '%Y-%m-%d').date()
    for j in range(len(removed_flaky)):
        if added_flaky['Test'].iloc[i] == removed_flaky['Test'].iloc[j]:
            remove_date = datetime.strptime(removed_flaky['Date'].iloc[j], '%Y-%m-%d').date()
            life_time = remove_date - add_date
            print(life_time, remove_date)
            life_time_df['Life_Time'].iloc[i] = life_time
            removed_flaky = removed_flaky.drop(removed_flaky.index[j])
            break

In [ ]:
add_df = df[df['Action'] == 'add']
remove_df = df[df['Action'] == 'remove']

In [ ]:
add_crash_df = add_df[add_df['Expectation'] == 'crash']
add_failure_df = add_df[add_df['Expectation'] == 'failure']
add_timeout_df = add_df[add_df['Expectation'] == 'timeout']
add_skip_df = add_df[add_df['Expectation'] == 'skip']
flaky_outcomes = ['failure pass', 'pass timeout','failure pass timeout', 'crash pass','crash failure pass timeout', 
                  'crash failure pass', 'failure pass skip timeout', 'pass skip timeout', 'crash pass timeout',
                  'crash failure pass skip timeout', 'crash pass skip timeout']
add_flaky_agg = add_df[add_df['Expectation'].isin(flaky_outcomes)]

In [ ]:
result_df = pd.DataFrame(columns=['Outcome', 'Count', 'Cross-Platform-Count', 'Platform-Specific-Count', 'BugId-Assigned-Count', 'BugId-Assigned-PErcentage', 'Cross-Platform-Percentage'])

In [ ]:
def get_stats(outcome_type, outcome_add_df = add_flaky_agg):
    if type(outcome_type) == str:
        outcome_add_df = add_df[add_df['Expectation'] == outcome_type]
    outcome_add_df_bug = outcome_add_df[outcome_add_df['Bug_Id'].isna()]
    outcome_add_df_cross_platform = outcome_add_df[outcome_add_df['Build_type'].isna()]
    if outcome_type == 0:
        outcome_type = 'flaky'
    row = {'Outcome': outcome_type,
           'Count': len(outcome_add_df),
           'Cross-Platform-Count': len(outcome_add_df_cross_platform),
           'Platform-Specific-Count': len(outcome_add_df) - len(outcome_add_df_cross_platform),
           'BugId-Assigned-Count': len(outcome_add_df) - len(outcome_add_df_bug),
           'BugId-Assigned-PErcentage': (len(outcome_add_df) - len(outcome_add_df_bug)) / len(outcome_add_df),
           'Cross-Platform-Percentage': (len(outcome_add_df_cross_platform)) / len(outcome_add_df)
          }
    return row

In [ ]:
result_df = result_df.append(get_stats('failure'), ignore_index = True)
result_df = result_df.append(get_stats(0), ignore_index = True)
result_df = result_df.append(get_stats('timeout'), ignore_index = True)
result_df = result_df.append(get_stats('skip'), ignore_index = True)
result_df = result_df.append(get_stats('crash'), ignore_index = True)
result_df